In [1]:
import sys
from pyspark.sql.window import Window
import pyspark.sql.functions as func
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.sql import Row

In [7]:
spark = SparkContext("local", "sqlContext")
sql = SQLContext(spark)

In [8]:
data = [('Thin', 'Cell Phone', 6000), ('Normal', 'Tablet', 1500), ('Mini', 'Tablet' , 5500), ('Ultra Thin', 'Cell Phone', 5000), ('Very Thin', 'Cell Phone', 6000), ('Big', 'Tablet', 2500), ('Bendable', 'Cell Phone', 3000), ('Foldable', 'Cell Phone', 3000), ('Pro', 'Tablet', 4500),  ('Pro2', 'Tablet', 6500)]
df = sql.createDataFrame(data, ['product', 'category', 'revenue'])
df.show()

+----------+----------+-------+
|   product|  category|revenue|
+----------+----------+-------+
|      Thin|Cell Phone|   6000|
|    Normal|    Tablet|   1500|
|      Mini|    Tablet|   5500|
|Ultra Thin|Cell Phone|   5000|
| Very Thin|Cell Phone|   6000|
|       Big|    Tablet|   2500|
|  Bendable|Cell Phone|   3000|
|  Foldable|Cell Phone|   3000|
|       Pro|    Tablet|   4500|
|      Pro2|    Tablet|   6500|
+----------+----------+-------+



In [9]:
sql.registerDataFrameAsTable(df, 'productRevenue')

In [12]:
spec = Window.partitionBy(df['category']).orderBy(df['revenue'].desc()).rangeBetween(-sys.maxsize, sys.maxsize)
df_pr = sql.table("productRevenue")
diff = (func.max(df_pr['revenue']).over(spec) - df_pr['revenue'])
df_sel = df_pr.select(df_pr['product'], df_pr['category'], df_pr['revenue'], diff.alias('diff'))
df_sel.show()

+----------+----------+-------+----+
|   product|  category|revenue|diff|
+----------+----------+-------+----+
|      Pro2|    Tablet|   6500|   0|
|      Mini|    Tablet|   5500|1000|
|       Pro|    Tablet|   4500|2000|
|       Big|    Tablet|   2500|4000|
|    Normal|    Tablet|   1500|5000|
|      Thin|Cell Phone|   6000|   0|
| Very Thin|Cell Phone|   6000|   0|
|Ultra Thin|Cell Phone|   5000|1000|
|  Bendable|Cell Phone|   3000|3000|
|  Foldable|Cell Phone|   3000|3000|
+----------+----------+-------+----+



In [6]:
spark.stop()